# [8] 데이터 준비하기 ; 조인, 병합, 변형       

* 8.1 계층적 색인
    * 8.1.1 계층의 순서를 바꾸고 정렬하기
    * 8.1.2 계층별 요약 통계
    * 8.1.3 DataFrame의 컬럼 사용하기
* 8.2 데이터 합치기
    * 8.2.1 데이터베이스 스타일로 DataFrame 합치기
    * 8.2.2 색인 병합하기
    * 8.2.3 축 따라 이어 붙이기
    * 8.2.4 겹치는 데이터 합치기
* 8.3 재형성과 피벗
    * 8.3.1 계층적 색인으로 재형성 하기
    * 8.3.2 긴 형식에서 넓은 형식으로 피벗하기
    * 8.3.3 넒은 형식에서 긴 형식으로 피벗하기
---
---



In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## [8.1] 계층적 색인
---
> 축에 대해 다중(둘 이상) 색인 단계를 지정할 수 있도록 해준다   
* stack() : Series -> DataFrame
* unstack() : DataFrame -> Series           
[stack vs unstack](https://pandas.pydata.org/docs/user_guide/reshaping.html)

In [3]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

> 계층적으로 색인된 객체는 데이터의 부분집합을 `부분적 색인으로 접근`하는 것이 가능하다.

In [5]:
data['b']
data['b':'c']
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [6]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

> unstack 메서드를 사용해서 데이터를 새롭게 배열할 수 있다. (Series -> DataFrame 객체로 변환)

In [7]:
print(data)
print("-------------------------------")
data.unstack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64
-------------------------------


,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [ ]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

> DataFrame에서는 두 축 모두 계층적 색인을 가질 수 있다.

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

> 계층적 색인의 각 단계는 이름을 가질 수 있고, 만약 이름을 가지고 있다면 콘솔 출력시 함께 나타난다.

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

> MultiIndex는  따로 생성한 다음에 재사용이 가능하다.    
> MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

### [8.1.1] 계층의 순서를 바꾸고 정렬하기     

> swaplevel 은 넘겨 받은 두개의 계층 번호나 이름이 뒤바뀐 새로운 객체를 반환한다.(데이터는 변하지 않음)



In [ ]:
print(frame)
print("------------------------------------------")
print(frame.swaplevel('key1', 'key2'))

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11
------------------------------------------
state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11


> 

> sort_index 단일 계층에 속한 데이터를 정렬한다.

In [ ]:
print(frame.sort_index(level=1))
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### [8.1.2] 계층별 요약 통계        
DataFrame과  Series의 많은 기술 통계와 요약 통계는 "level" 옵션을 가지고 있는데, 어떤 한 축에 대해 합을 구하고 싶은 단계를 지정할 수 있는 옵션이다.

In [ ]:
print(frame)
print("----------------------------")
print(frame.sum(level='key2'))
print("----------------------------")
frame.sum(level='color', axis=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11
----------------------------
state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16
----------------------------


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### [8.1.3] DataFrame의 컬럼 사용하기

> set_index(), reset_index()

In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [ ]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [ ]:
print(frame.set_index(['c', 'd'], drop=True))
print("-----------------------------------------------")
print(frame.set_index(['c', 'd'], drop=False))

       a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1
-----------------------------------------------
       a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3


> reset_index 함수는 set_index와 반대되는 개념인데 계층적 색인 단계가 컬럼으로 이동한다.

In [ ]:
print(frame2)
print("------------------------------")
frame2.reset_index()

       a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1
------------------------------


,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## [8.2] 데이터 합치기
---

pandas 객체에 저장된 데이터는 여러 가지 방법으로 합칠 수 있다.
* pandas.merge 는 하나 이상의 키를 기준으로 DataFrame의 로우를 합칠 수 있다. 관계형 데이터베이스의 join과 비슷하다.   
* pandas.concat은 하나의 축을 따라 객체를 이어붙인다.   
* combile_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

### [8.2.1] 데이터베이스 스타일로 DataFrame 합치기

> [내부 조인 대 외부 조인 : 예제와의 정확한 차이](https://ko.myservername.com/inner-join-vs-outer-join)

merge 함수는 기본적으로 내부 조인을 수행하므로 교집합인 결과를 반환한다.

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
df2

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6


,key,data2
0,a,0
1,b,1
2,d,2


In [ ]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


merge 함수는 중복된 컬럼 이름을 키(예제 key 컬럼)로 사용한다. 하지만 명시적으로 지정해주는 습관을 들이는게 좋다.

In [ ]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


만약 두 객체에 중복된 컬럼 이름이 없으면 따로 지정해주면 된다.

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df3)
print(df4)
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


> ***how 인자를 'left', 'right', 'outer'를 넘겨서 각각 왼쪽 조인, 오른쪽 조인, 외부 조인을 수행할 수 있다.           

In [ ]:
print(pd.merge(df1, df2, how='inner')) # default
print("----------------------------------")
print(pd.merge(df1, df2, how='left'))
print("----------------------------------")
print(pd.merge(df1, df2, how='right'))
print("----------------------------------")
print(pd.merge(df1, df2, how='outer'))

  key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0
----------------------------------
  key  data1  data2
0   b      0    1.0
1   b      1    1.0
2   a      2    0.0
3   c      3    NaN
4   a      4    0.0
5   a      5    0.0
6   b      6    1.0
----------------------------------
  key  data1  data2
0   a    2.0      0
1   a    4.0      0
2   a    5.0      0
3   b    0.0      1
4   b    1.0      1
5   b    6.0      1
6   d    NaN      2
----------------------------------
  key  data1  data2
0   b    0.0    1.0
1   b    1.0    1.0
2   b    6.0    1.0
3   a    2.0    0.0
4   a    4.0    0.0
5   a    5.0    0.0
6   c    3.0    NaN
7   d    NaN    2.0


> 다대다 조인은 두 로우의 데카르트 곱을 반환한다.         
> 조인 메서드는 결과에 나타나는 구별되는 키에 대해서만 적용된다. (p.319)

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
print(df1, "\n------------------------------------------")
print(df2, "\n------------------------------------------")
pd.merge(df1, df2, on='key', how='left')

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5 
------------------------------------------
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4 
------------------------------------------


,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [ ]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


> 여러 개의 키를 병합하려면 컬럼 이름이 담긴 리스트를 넘기면 된다.

In [ ]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

print(left)
print("-----------------------------")
print(right)
print("-----------------------------")
pd.merge(left, right, on=['key1', 'key2'], how='outer')

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
-----------------------------
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7
-----------------------------


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


> 겹치는 컬럼에 대해서 축 이름을 변경해서 수동으로 컬럼 이름이 겹치게 할 수도 있고, merge 함수에 있는 suffixed 인자로 두 DataFrame 객체에서 겹치는 컬럼 이름 뒤에 붙일 문자열을 지정해줄 수 있다.

In [ ]:
print(pd.merge(left, right, on='key1'))
print("-------------------------------------")
print(pd.merge(left, right, on='key1', suffixes=('_left', '_right')))

  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7
-------------------------------------
  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7


### [8.2.2] 색인 병합하기
---
> 병합하려는 키가 DataFrame의 색인일 경우가 있다. 이런 경우에는 left_index=True 혹은 right_index=True 옵션(혹은 둘다)을 지정해서 해당 색인을 병합키로 사용할 수 있다.

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
print("------------------------------")
print(right1)
print("------------------------------")
pd.merge(left1, right1, left_on='key', right_index=True)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
------------------------------
   group_val
a        3.5
b        7.0
------------------------------


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') # outer는 합집함. Default 는 how='inner' 교집합

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002], 'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]], columns=['event1', 'event2'])
print(lefth)
print('-----------------------------------------------')
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
-----------------------------------------------
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [ ]:
print(pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True))
print('-----------------------------------------------')
print(pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer'))

     key1  key2  data  event1  event2
0    Ohio  2000   0.0       4       5
0    Ohio  2000   0.0       6       7
1    Ohio  2001   1.0       8       9
2    Ohio  2002   2.0      10      11
3  Nevada  2001   3.0       0       1
-----------------------------------------------
     key1  key2  data  event1  event2
0    Ohio  2000   0.0     4.0     5.0
0    Ohio  2000   0.0     6.0     7.0
1    Ohio  2001   1.0     8.0     9.0
2    Ohio  2002   2.0    10.0    11.0
3  Nevada  2001   3.0     0.0     1.0
4  Nevada  2002   4.0     NaN     NaN
4  Nevada  2000   NaN     2.0     3.0


In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2
right2
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [ ]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [ ]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
print(another)
print("---------------------------------------------------------")
print(left2.join([right2, another]))
print("---------------------------------------------------------")
left2.join([right2, another], how='outer')

   New York  Oregon
a       7.0     8.0
c       9.0    10.0
e      11.0    12.0
f      16.0    17.0
---------------------------------------------------------
   Ohio  Nevada  Missouri  Alabama  New York  Oregon
a   1.0     2.0       NaN      NaN       7.0     8.0
c   3.0     4.0       9.0     10.0       9.0    10.0
e   5.0     6.0      13.0     14.0      11.0    12.0
---------------------------------------------------------


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### [8.2.3] 축 따라 붙히기

> 데이터를 합치는 방법은 이어붙이기가 있다.   
> Numpy는 ndarray를 이어붙이는 concatenate 함수를 제공한다.            
> Pandas 에서는 concat 함수를 사용한다.
> join_aex 속성 0.25 버전 ???????

> * 고려 사항
> 1. 만약 연결하려는 두 객체의 색인이 서로 다르면 그 색인의 교집합이어야 하는가 아니면 합집합이여야 하는가   
> 2. 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있어야 하는가?
> 3. 어떤 축으로 연결할 것인지 고려 해야 하는가? 많은 경우 DataFrame의 기본 정수 라벨이 가장 먼저 무시 된다. 

In [ ]:
arr = np.arange(12).reshape((3, 4))
print(arr,"\n------------------------------------------")
print(np.concatenate([arr, arr]), "\n------------------------------------------")
np.concatenate([arr, arr], axis=1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]] 
------------------------------------------
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]] 
------------------------------------------


array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
print(s1, "\n------------------------------------------")
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
print(s2, "\n------------------------------------------")
s3 = pd.Series([5, 6], index=['f', 'g'])
print(s3)

a    0
b    1
dtype: int64 
------------------------------------------
c    2
d    3
e    4
dtype: int64 
------------------------------------------
f    5
g    6
dtype: int64


> axis = 0 이 default

In [ ]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

> axis =1 로 붙이면 Series 객체 -> DataFrame 객체로 변경된다.

In [ ]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


> 겹치는 축이 없기 때문에 외부 조인으로 정렬된 합집합을 얻었지만 join='inner' 를 넘겨서 교집합을 구할 수도 있다.

In [ ]:
s4 = pd.concat([s1, s3])
print(s1)
print("----------------")
print(s4)
print("----------------")
print(pd.concat([s1, s4], axis=1))
print("----------------")
pd.concat([s1, s4], axis=1, join='inner')

a    0
b    1
dtype: int64
----------------
a    0
b    1
f    5
g    6
dtype: int64
----------------
     0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6
----------------


,0,1
a,0,0
b,1,1


> 병합하려는 축 직접 지정

In [ ]:
print(pd.__version__) # 0.25 버전에서 사용 가능;;;
#pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])
pd.concat([s1, s4], axis=1).reindex(['a', 'c', 'b', 'e'])


1.1.5


,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
print(result)
result.unstack()

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64


,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [ ]:
print(pd.concat([s1, s2, s3], axis=1))
print("--------------------------")
print(pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three']))

     0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0
--------------------------
   one  two  three
a  0.0  NaN    NaN
b  1.0  NaN    NaN
c  NaN  2.0    NaN
d  NaN  3.0    NaN
e  NaN  4.0    NaN
f  NaN  NaN    5.0
g  NaN  NaN    6.0


> DataFrame 객체에 대해서도 지금까지와 같은 방식으로 적용할 수 있다

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
print(df1, "\n-----------------------------------")
print(df2, "\n-----------------------------------")
pd.concat([df1, df2], axis=0, keys=['level1', 'level2'])

   one  two
a    0    1
b    2    3
c    4    5 
-----------------------------------
   three  four
a      5     6
c      7     8 
-----------------------------------


one  two  three  four
level1 a  0.0  1.0    NaN   NaN
       b  2.0  3.0    NaN   NaN
       c  4.0  5.0    NaN   NaN
level2 a  NaN  NaN    5.0   6.0
       c  NaN  NaN    7.0   8.0

> 리스트 대신 객체의 사전을 넘기면 사전의 키가 keys 옵션으로 사용 된다.

In [ ]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

> 새로 생성된 계층의 이름은 names 인자로 지정할 수 있다.

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower']) # axis=1 이기 때문에 리스트의 0번이 column 의 name 으로 들어간다.

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

> DataFrame의 로우 색인이 분석에 필요한 데이터를 포함하고 있지 않은 경우     
> ignore_index = True 옵션을 주면 된다. ?????????????(p.331)

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

print(df1,"\n-----------------------------------")
print(df2)

          a         b         c         d
0  1.246435  1.007189 -1.296221  0.274992
1  0.228913  1.352917  0.886429 -2.001637
2 -0.371843  1.669025 -0.438570 -0.539741
          b         d         a
0  0.476985  3.248944 -1.021228
1 -0.577087  0.124121  0.302614


In [ ]:
pd.concat([df1, df2], ignore_index=False) # defalut
print('--------------------------')
pd.concat([df1, df2], ignore_index=True)

--------------------------


,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


### [8.2.4]겹치는 데이터 합치기(p.332)

> 데이터를 합칠 때 병합이나 이어붙이기로는 불가능한 상황이 있는데, 두 데이터 셋의 색인이 일부 겹치거나 전체가 겹치는 경우가 그렇다.    
> 벡터화된 if-else 구문을 표현하는 Numpy의 where함수로 자세히 알아보자.

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print(a)
print("----------------")
print(b)
print("----------------")
np.where(pd.isnull(a), b, a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
----------------
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64
----------------


array([0. , 2.5, 2. , 3.5, 4.5, nan])

> combine_first() 는 위와 동일 연산과 데이터 정렬 기능까지 제공한다.                
> error 발생 ??????????????????????

In [ ]:
print(b[:-2])
print(a[2:])
b[:-2].combine_first(a[2:])

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64


TypeError: ignored

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

## [8.3] 재형성과 피벗
---------

### [8.3.1] 계층적 색인으로 재형성하기
* stack : 데이터의 컬럼을 로우로 피벗(회전)시킨다.
* unstack : 로우를 컬럼으로 피벗시킨다.

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [ ]:
result = data.stack()
result# Series 객체를 반환

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [ ]:
result.unstack() # DataFrame 객체를 반환

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


> 기본적으로 가장 안쪽에 있는 레벨부터 끄집어 내는데(stack도 마찬가지) 레벨숫자나 이름을 전달해서 끄집어낼 단계를 지정할 수 잇다.

In [ ]:
print(result)
print("------------------")
print(result.unstack(0)) #
print("------------------")
print(result.unstack('state'))

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64
------------------
state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5
------------------
state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5


> 해당 레벨에 있는 모든 값이 하위 그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길 수 있다.

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
print(s1)
print("-------------------")
print(s2)
print("-------------------")
data2 = pd.concat([s1, s2], keys=['one', 'two'])
print(data2)
print("-------------------")
print(data2.unstack())

a    0
b    1
c    2
d    3
dtype: int64
-------------------
c    4
d    5
e    6
dtype: int64
-------------------
one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64
-------------------
       a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0


> stack() 함수는 누락 데이터를 알아서 걸러준다.

In [ ]:
print(data2.unstack())
print("-------------------")
print(data2.unstack().stack())
print("-------------------")
print(data2.unstack().stack(dropna=False)) # default : True

       a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0
-------------------
one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64
-------------------
one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64


In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df
df.unstack('state')

TypeError: ignored

In [ ]:
df.unstack('state').stack('side')

NameError: ignored

### [8.3.2]긴 형식에서 넓은 형식으로 피벗하기       

* pivot() : 하나의 컬럼을 여러개의 새로운 DataFrame으로 생성
* melt() : 여러 컬럼을 하나의 병합하고 Data Frame을 입력보다 긴형태로 만들어낸다

[pivot vs melt](https://pandas.pydata.org/docs/user_guide/reshaping.html)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/examples/macrodata.csv')

print(data.head())
print("-----------------------------------")
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
print(periods)
print("-----------------------------------")
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
print(data)
print("-----------------------------------")
data.index = periods.to_timestamp('D', 'end')
print(data)
print("-----------------------------------")
ldata = data.stack().reset_index().rename(columns={0: 'value'})

   year  quarter   realgdp  realcons  ...  unemp      pop  infl  realint
0  1959        1  2710.349    1707.4  ...    5.8  177.146  0.00     0.00
1  1959        2  2778.801    1733.7  ...    5.1  177.830  2.34     0.74
2  1959        3  2775.488    1751.8  ...    5.3  178.657  2.74     1.09
3  1959        4  2785.204    1753.7  ...    5.6  179.386  0.27     4.06
4  1960        1  2847.699    1770.5  ...    5.2  180.007  2.31     1.19

[5 rows x 14 columns]
-----------------------------------
PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')
-----------------------------------
item    realgdp  infl  unemp
0      2710.349  0.00    5.8
1      2778.801  2.34    5.1
2      2775.488  2.74    5.3
3      

In [ ]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,1.246435
1,1959-03-31 23:59:59.999999999,infl,0.000,1.007189
2,1959-03-31 23:59:59.999999999,unemp,5.800,-1.296221
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.274992
4,1959-06-30 23:59:59.999999999,infl,2.340,0.228913
5,1959-06-30 23:59:59.999999999,unemp,5.100,1.352917
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.886429
7,1959-09-30 23:59:59.999999999,infl,2.740,-2.001637
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.371843
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,1.669025


In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value            ...    value2          
item                           infl   realgdp  ...   realgdp     unemp
date                                           ...                    
1959-03-31 23:59:59.999999999  0.00  2710.349  ...  1.246435 -1.296221
1959-06-30 23:59:59.999999999  2.34  2778.801  ...  0.274992  1.352917
1959-09-30 23:59:59.999999999  2.74  2775.488  ...  0.886429 -0.371843
1959-12-31 23:59:59.999999999  0.27  2785.204  ...  1.669025 -0.539741
1960-03-31 23:59:59.999999999  2.31  2847.699  ...  0.476985 -1.021228
1960-06-30 23:59:59.999999999  0.14  2834.390  ... -0.577087  0.302614
1960-09-30 23:59:59.999999999  2.70  2839.022  ...  0.523772  1.343810

[7 rows x 6 columns]

### [8.3.3] 넓은 형식에서 긴 형식으로 피벗하기

In [ ]:
df = pd.DataFrame({'hey': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,hey,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
melted = pd.melt(df, ['hey'])
melted

,hey,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
reshaped = melted.pivot('hey', 'variable', 'value') # melted.pivot(index='hey', variable='variable', value='value')
reshaped

variable,A,B,C
hey,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [ ]:
reshaped.reset_index()

variable,hey,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [ ]:
pd.melt(df, id_vars=['hey'], value_vars=['A', 'B'])

,hey,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['hey', 'A', 'B'])

,variable,value
0,hey,foo
1,hey,bar
2,hey,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
